<a href="https://www.kaggle.com/code/raxmatillo/aviachiptabashorat?scriptVersionId=219023960" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv
/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv


# Ma'lumotlarni tayyorlab olamiz

In [2]:
train = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv')
test = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv')
sample = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv')
train.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [3]:
train.describe()

,id,duration,days_left,price
count,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,12.177627,25.924150,20960.281700
std,5773.647028,7.157944,13.624874,22775.459535
min,1.000000,0.830000,1.000000,1105.000000
25%,5000.750000,6.830000,14.000000,4783.000000
50%,10000.500000,11.250000,26.000000,7425.000000
75%,15000.250000,16.080000,38.000000,42521.000000
max,20000.000000,38.580000,49.000000,114523.000000


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                20000 non-null  int64  
 1   airline           20000 non-null  object 
 2   flight            20000 non-null  object 
 3   source_city       20000 non-null  object 
 4   departure_time    20000 non-null  object 
 5   stops             20000 non-null  object 
 6   arrival_time      20000 non-null  object 
 7   destination_city  20000 non-null  object 
 8   class             20000 non-null  object 
 9   duration          20000 non-null  float64
 10  days_left         20000 non-null  int64  
 11  price             20000 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 1.8+ MB


In [5]:
train['class'] = train['class'].map({'Economy': 0, 'Business': 1})
train['stops'] = train['stops'].map({'one': 1, 'zero': 0, 'two_or_more': 2})

In [6]:
# o'rtacha qimmat va o'rtacha arzon aviakompaniyalar
train.groupby(['airline']).agg({'price': 'mean'})
# ['AirAsia', 'Indigo', 'GO_FIRST','SpiceJet','Air_India', 'Vistara']

,price
airline,
AirAsia,4149.467803
Air_India,23622.449637
GO_FIRST,5683.032493
Indigo,5339.213748
SpiceJet,6059.975248
Vistara,30473.743409


In [7]:
train['airline'] = train['airline'].map({'AirAsia': 0, 'Indigo': 1, 'GO_FIRST': 2, 'SpiceJet': 3, 'Air_India': 4, 'Vistara': 5})
train.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,5,UK-810,Bangalore,Early_Morning,1,Night,Mumbai,0,14.25,21,7212
1,2,3,SG-5094,Hyderabad,Evening,0,Night,Kolkata,0,1.75,7,5292
2,3,5,UK-846,Bangalore,Morning,1,Evening,Delhi,1,9.58,5,60553
3,4,5,UK-706,Kolkata,Morning,1,Evening,Hyderabad,0,6.75,28,5760
4,5,1,6E-5394,Chennai,Early_Morning,0,Morning,Mumbai,0,2.00,4,10712


In [8]:
train.select_dtypes(include=(int, float)).corrwith(train['price']).abs()
# train.hist(bins=50, figsize=(12,8))
# sns.countplot(train)

id           0.002443
airline      0.455374
stops        0.121455
class        0.937868
duration     0.213158
days_left    0.102545
price        1.000000
dtype: float64

In [9]:
train.isnull().sum()

id                  0
airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64

In [10]:
train.shape

(20000, 12)

In [11]:
train.select_dtypes(include='object').columns

Index(['flight', 'source_city', 'departure_time', 'arrival_time',
       'destination_city'],
      dtype='object')

In [12]:
train.select_dtypes(include='number').columns

Index(['id', 'airline', 'stops', 'class', 'duration', 'days_left', 'price'], dtype='object')

# Machine Learningga tayyorlaymiz

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

X = train.drop('price', axis=1)
y = train['price']

scaler = StandardScaler()

nums = ['id', 'airline', 'stops', 'class', 'duration', 'days_left']
cats = ['flight', 'source_city', 'departure_time', 'arrival_time','destination_city']

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')


full_pipeline = ColumnTransformer(
    transformers=[
        ('num', num_transformer, nums),  # Raqamli ustunlarni o'zgartirish
        ('cat', cat_transformer, cats)   # Kategorik ustunlarni o'zgartirish
    ])

# df = pd.get_dummies(df, columns=cats)
X_prepared = full_pipeline.fit_transform(X)
# X_prepared = X_prepared.toarray()
X_train, X_valid, y_train, y_valid = train_test_split(X_prepared, y, test_size=0.2, random_state=42)

In [14]:
X_train.toarray()
X_valid.toarray()

array([[ 1.12669905e-01, -9.61300866e-01,  1.92813824e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.37845264e+00,  8.45825492e-01,  1.92813824e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.30622565e-01,  8.45825492e-01,  1.92813824e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.66025405e-05,  2.43450039e-01,  1.92813824e-01, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-1.40807071e+00,  8.45825492e-01,  1.92813824e-01, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 9.00060203e-01, -3.58925414e-01, -2.30154354e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

# Baholash funksiyamiz

In [15]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score

def score(y_train, y_pred):
    mae = mean_absolute_error(y_train, y_pred)
    mse = mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    # EVS hisoblash
    evs = explained_variance_score(y_train, y_pred)
    print(f'MAE: {mae}')
    print(f'RMSE: {np.sqrt(mse)}')
    print(f'R-squared: {r2}')
    print(f"Explained Variance Score: {evs}")

# Bir nechta regressiya algoritmlari bilan model yozamiz

In [16]:
# LinearRegression
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression().fit(X_train, y_train)
lr_y_pred = lr_model.predict(X_valid)

print('LinearRegression (valid set)\n')
score(y_valid, lr_y_pred)

LinearRegression (valid set)

MAE: 4356.8467833895775
RMSE: 6344.773654519064
R-squared: 0.9233385409626872
Explained Variance Score: 0.9233797140448567


In [17]:
# RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_valid)

print('RandomForestRegressor:\n')
score(y_valid, rf_y_pred)

RandomForestRegressor:

MAE: 1702.7399175
RMSE: 3500.631466215299
R-squared: 0.976663399158021
Explained Variance Score: 0.9766635243595174


In [18]:
# DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor(random_state=42).fit(X_train, y_train)
tree_y_pred = tree_model.predict(X_valid)

print('DecisionTreeRegressor:\n')
score(y_valid, tree_y_pred)

DecisionTreeRegressor:

MAE: 1919.74975
RMSE: 4555.223654690294
R-squared: 0.9604847964349146
Explained Variance Score: 0.9604884688837456


In [19]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

# Modeldan bashorat qilish
y_pred_xgb = xgb_model.predict(X_valid)

# Natijalarni baholash
# XGB
print('XGBRegressor\n')
score(y_valid, y_pred_xgb)


XGBRegressor

MAE: 2158.4512075538637
RMSE: 3820.446885154817
R-squared: 0.9722045860789714
Explained Variance Score: 0.9722048024630646


# Test data bilan predict qilamiz

In [20]:
test['class'] = test['class'].map({'Economy': 0, 'Business': 1})
test['stops'] = test['stops'].map({'one': 1, 'zero': 0, 'two_or_more': 2})

test['airline'] = test['airline'].map({'AirAsia': 0, 'Indigo': 1, 'GO_FIRST': 2, 'SpiceJet': 3, 'Air_India': 4, 'Vistara': 5})

In [21]:
test_prepared = full_pipeline.transform(test)
test_prepared.toarray()

array([[-1.73196421,  0.24345004,  0.19281382, ...,  0.        ,
         0.        ,  0.        ],
       [-1.731791  ,  0.84582549,  0.19281382, ...,  0.        ,
         0.        ,  1.        ],
       [-1.7316178 ,  0.24345004, -2.30154354, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.86645842,  0.24345004,  0.19281382, ...,  0.        ,
         0.        ,  1.        ],
       [-0.86628521,  0.24345004,  0.19281382, ...,  1.        ,
         0.        ,  0.        ],
       [-0.86611201,  0.84582549,  0.19281382, ...,  0.        ,
         0.        ,  0.        ]])

In [22]:
y_pred = rf_model.predict(test_prepared)

# Natijani sample datasetining price ustuniga saqlaymiz

In [23]:
sample['price'] = y_pred

In [24]:
sample.head()

,id,price
0,1,51937.04
1,2,63761.98
2,3,23302.77
3,4,3215.31
4,5,6212.86


In [25]:
sample.to_csv('aviachiptabashorat_new.csv', index=False)

In [26]:
test

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,1,4,AI-765,Kolkata,Evening,1,Night,Delhi,1,28.25,2
1,2,5,UK-747,Delhi,Early_Morning,1,Night,Mumbai,1,13.83,34
2,3,4,AI-570,Mumbai,Early_Morning,0,Early_Morning,Chennai,1,2.00,30
3,4,0,I5-974,Hyderabad,Night,1,Late_Night,Delhi,0,5.17,26
4,5,4,AI-770,Kolkata,Night,1,Afternoon,Mumbai,0,16.33,35
...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,4,AI-768,Kolkata,Afternoon,1,Morning,Bangalore,1,17.42,15
4996,4997,1,6E-6214,Kolkata,Morning,0,Afternoon,Mumbai,0,3.00,40
4997,4998,4,AI-402,Kolkata,Morning,1,Night,Mumbai,1,11.17,37
4998,4999,4,AI-673,Mumbai,Early_Morning,1,Night,Hyderabad,1,13.33,38
